#Daugman Algorithm for Eye Recognition



In [280]:
## imports

import numpy as np
import cv2
import itertools
import math


In [281]:
def daugman(grey_image, center, min_radius, max_radius, step):
    
    intensities = []
    list_of_radiuses = list(range(min_radius, max_radius, step)) # list of radius rings we want to analyse
    mask = np.zeros_like(grey_image)
    for radius in list_of_radiuses:
        # drawing a circle on the mask
        cv2.circle(mask, center, radius, 255, 1)
        
        # extracting out the ring
        contrast = grey_image & mask
        
        total_intensity = np.sum(contrast.flatten())
        average_intensity = total_intensity / (2 * math.pi * radius) # finding the average intensity
        intensities.append(average_intensity)

        mask = np.zeros_like(grey_image) # restart mask
    
    
    intensities = np.array(intensities)
    
    # finding the difference between each intensity ring
    intensities_diff = np.diff(intensities)
    
    intensities_diff = abs(cv2.GaussianBlur(intensities_diff, (1, 5), 0))


    index = np.argmax(intensities_diff) # finding the index of the maximum difference in intensity
    
    return intensities_diff[index], list_of_radiuses[index]
        
        
        
    
    

In [282]:
def iris(grey_image, min_radius, max_radius, step_coords, step_radius):
    
    height, width = grey_image.shape
    if height != width:
        # crop image into a square
        if height < width:
            grey_image = grey_image[:, int((width-height)/2): width-int((width-height)/2)-1]
        else:
            grey_image = grey_image[int((height-width)/2): height-int((height-width)/2)-1,:]
        
    height, width = grey_image.shape
        
    # let us assume that the iris is in the centre 1/3 of the image
    
    # list of coordinates that are inside the centre 1/3 of the image...
    one_third_x = list(range(int(height/3), int(height/3 * 2), step_coords))
    centre_third_square = itertools.product(one_third_x, one_third_x) # collecting coordinates in the centre 1/3
    intensity = []
    coords = []
    
    for pt in centre_third_square:
        intenseness, radius = daugman(grey_image, pt, min_radius, max_radius, step_radius)
        intensity.append(intenseness)
        coords.append((pt, radius))
        
    
    intensity = np.array(intensity)
    max_index = np.argmax(intensity)

    return coords[max_index]
    
    

In [283]:
image = cv2.imread('eye2.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

iris(image, 15, 100, 1, 3)


((58, 57), 36)

## Explanation

The algorithm used to find the centre coordinates and the radius of the iris and pupil is Daugman. The operator searches for the circular path where there is maximum change in pixel values, varying the raiuds and the centre of the contour. Ths is applied iteratively, only for the centre 1/3 of the image as we assume the pupil is in the centre of the image.

A Gaussian filter is applied to smooth the images of the eye in order to remove noise.

## Further Improvements

This is still a relatively basic implementation of the Daugman algorithm. In pictures of eyes, there can sometimes be [a light reflection that is circular in nature](https://www.istockphoto.com/photo/eye-xxl-gm108220711-7593226). This light spots increase the intensity of the image and results in noisiness.

To counter this, we can use an operator to fill in the light-affected regions with the average intensities of pixels from the region surrounding it. The image can be modified using MATLAB imfill before being processed. 

Additionally, the iris border is expected to lie completely inside the image. For every point that we iterate through, the sum of values inside the radius ring is noted. Hence, those which has rings that lie outside of the image dimension can be neglected. 
